# Import material from JARVIS database

<h2 style="color:green">Usage</h2>

1. Install `express-py` alongside the necessary packages
2. Import the material from JARVIS database
3. Convert the material to ESSE format
4. Pass the material to Materials Designer runtime




# Create an interface between two materials with minimal strain

Import a structure from the NIST JARVIS database by its ID. Use search at <a href="https://jarvis.nist.gov/jarvisdft/" target="_blank">JARVIS-DFT</a> to find the ID of the structure you are interested in.

<h2 style="color:green">Usage</h2>

2. Set Input Parameters (e.g. `JARVIS_ID`) below or use the default value
3. Click "Run" > "Run All" to run all cells
4. Wait for the run to complete (it should take ~30 sec first time to install packages). Scroll down to view cell results.

## Methodology

The following happens in the script below:

1. Install `express-py` alongside the necessary packages
2. Import the material from JARVIS database
3. Convert the material to ESSE format
4. Pass the material to the outside runtime

## 1. Set Parameters

In [ ]:
# Note: JVASP-670 is an entry for MoTe2
JARVIS_IDS = [
    "JVASP-670",
]

## 2. Install Packages

In [ ]:
from jupyterlite.utils import install_packages

await install_packages("import_material_from_jarvis_db_entry.ipynb")

## 3. Data Processing

### 3.1. Get data and wrap it in dataframe.

In [ ]:
import pandas as pd
from jarvis.db.figshare import data

dft_2d = data("dft_2d")
dataframe = pd.DataFrame(dft_2d)

### 3.2. Filter data

In [ ]:
import json
from express import ExPrESS

dataframe_entry_for_jarvis_id = dataframe[(dataframe["jid"].isin(JARVIS_IDS))]
dataframe_entry_as_json = dataframe_entry_for_jarvis_id.to_json(orient="records")
jarvis_db_entry = json.dumps(json.loads(dataframe_entry_as_json)[0])


### 3.3. Convert to ESSE format

In [ ]:
kwargs = {
    "structure_string": jarvis_db_entry,
    "cell_type": "original",
    "structure_format": "jarvis-db-entry",
}

handler = ExPrESS("structure", **kwargs)
data = handler.property("material", **kwargs)

### 3.4. Preview the data

In [ ]:
print(json.dumps(data, indent=4))

## 4. Pass data to the outside runtime

In [ ]:
from jupyterlite.utils import set_data

output_materials = [data]
set_data("materials", output_materials)